In [2]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [3]:
driver = webdriver.Chrome('/usr/bin/chromedriver')

In [4]:
all_data = []
for pages in range(1,4):
    """
    it was discovered that for each page,only one part of the url is changing, i am going to modifiying the url using f string
    one can change pages by clicking the pagination at the bottom 
    """
    url = f'https://hfr.health.gov.ng/facilities/hospitals-search?_token=4Wll44OzOr1kZWrvOzm7FGC1y3zCYbgGs99vHRSf&state_id=124&ward_id=0&facility_level_id=0&ownership_id=0&operational_status_id=1&registration_status_id=2&license_status_id=1&geo_codes=0&service_type=0&service_category_id=0&entries_per_page=20&page={pages}'
    driver.get(url)
    time.sleep(5)
    
    #gets the source code of the table
    hosp = driver.find_element_by_id('hosp').get_attribute('innerHTML')
    
    #parse the code
    soup = bs(hosp,'lxml')
    
    #find all the rows
    rows = soup.find_all('tr')
    
    #the first element is the cloumns while the others are rows
    row = rows[1:]
    
    for j,i in enumerate(row):
        details = [a.text for a in i.find_all('td')[:-1]]
        Facility_UID = details[3]
        xpath = f'//*[@id="hosp"]/tbody/tr[{j+1}]/td[9]/a/button'
        #just in case it didn't close the pop up the first time
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        time.sleep(5)
        temp_all_data = [[Facility_UID]]
        #sometimes, the click does not work the first time and fails twice too, that is while i used for loop
        for i in range(5):
            try:
                driver.find_element_by_xpath(xpath).click()
            except:
                pass



        html = driver.find_element_by_xpath('//*[@id="view_details"]/div').get_attribute('innerHTML')
        soup = bs(html,'lxml')   
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        panel = soup.find_all('div',{'class':'panel panel-default'})

        for k in panel:
            each_row = k.find_all('div',{'class':'row'})
            data = []
            for l in each_row:
                data.append(l.find('div').text)
            temp_all_data.append(data)
        all_data.append(temp_all_data)

In [5]:
len(all_data)

60